In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import os
import sys
import torch
import hydra
from pathlib import Path
from lightning import Fabric

# 设置项目路径，导入项目相关模块
project_root = Path(os.getcwd()).parent
sys.path.insert(0, str(project_root))

from gnf_visualizer import setup_environment, load_config, load_model, create_converter, prepare_data, visualize_reconstruction, visualize_1d_gradient_field_comparison
from funcmol.utils.utils_nf import load_neural_field, get_latest_model_path

# 设置输出目录
output_dir = "gnf_reconstruction_results"
os.makedirs(output_dir, exist_ok=True)

# 获取最新的模型文件夹
model_path = get_latest_model_path(
    base_exp_dir=str(Path.cwd().parent / "exps" / "neural_field"),
    model_prefix="nf_qm9_"
)
# TODO：也可以手动指定
model_dir = "/home/huayuchen/funcmol-main-neuralfield/funcmol/exps/neural_field/nf_qm9_20250803_150822_593945"

Using latest model from: /home/huayuchen/funcmol-main-neuralfield/funcmol/exps/neural_field/nf_qm9_20250801_223228_271628


In [37]:
# 初始化环境
fabric = Fabric(
    accelerator="auto",
    devices=1,
    precision="32-true",
    strategy="auto"
)
fabric.launch()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

config_dir = str(Path.cwd().parent / "configs")
with hydra.initialize_config_dir(config_dir=config_dir, version_base=None):
    config = hydra.compose(config_name="train_nf_qm9")

# 在这里修改数据路径
config["dset"]["data_dir"] = str(Path.cwd().parent / "dataset" / "data")

# 准备数据
batch, gt_coords, gt_types = prepare_data(fabric, config, device)

print(f"\nProcessing model from: {model_dir}")

# 加载指定模型
encoder, decoder = load_model(fabric, config, model_dir=model_dir)
converter = create_converter(config, device)

# 生成 codes
with torch.no_grad():
    codes = encoder(batch)

>> val set size: 20042
Input coords range: X[-3.129, 3.129], Y[-3.157, 3.157], Z[-3.537, 3.537]

Processing model from: /home/huayuchen/funcmol-main-neuralfield/funcmol/exps/neural_field/nf_qm9_20250803_150822_593945
Loading model from: /home/huayuchen/funcmol-main-neuralfield/funcmol/exps/neural_field/nf_qm9_20250803_150822_593945/model.pt
>> loaded dec
>> loaded enc
>> loaded dec
>> loaded enc
Model loaded successfully!


In [38]:
results = visualize_reconstruction(
    gt_coords=gt_coords,
    gt_types=gt_types,
    converter=converter,
    decoder=decoder,
    codes=codes,
    output_dir=output_dir,
    sample_idx=1, # TODO：这里可以改
    field_option="predicted_field", # 可选："predicted_field"、"gt_field"、"gt_field_with_noise"
    gradient_field_method="gaussian_mag"  # 可选： softmax, sfnorm, logsumexp, inverse_square, tanh, gaussian_mag, distance"
)
print(f"  GIF: {results['gif_path']}")
print(f" 对比图: {results['comparison_path']}")


Starting reconstruction for molecule 1
Ground truth atoms: 9
Atom types: [0, 2, 0, 1, 1, 1, 1, 1, 1]
Ground truth coords range: X[-1.120, 1.120], Y[-1.726, 1.726], Z[-1.481, 1.481]
Initial point range: X[-2.240, 2.240], Y[-3.452, 3.452], Z[-2.962, 2.962]
Initial points for atom type 0: Z[-2.954, 2.949]
Initial points for atom type 1: Z[-2.958, 2.955]
Initial points for atom type 2: Z[-2.950, 2.959]
Initial points for atom type 3: Z[-2.959, 2.961]
Initial points for atom type 4: Z[-2.956, 2.960]
Atom type 0: Mean Z gradient magnitude = 0.007376
Atom type 1: Mean Z gradient magnitude = 0.014499
Atom type 2: Mean Z gradient magnitude = 0.004062
Atom type 3: Mean Z gradient magnitude = 0.000413
Atom type 4: Mean Z gradient magnitude = 0.000366
Iteration 0, Atom type 0: grad norm = 0.013956, Z range = 5.903000
Atom type 0: Mean Z gradient magnitude = 0.006675
Atom type 1: Mean Z gradient magnitude = 0.016840
Atom type 2: Mean Z gradient magnitude = 0.002565
Atom type 3: Mean Z gradient mag

Atom type 0: Mean Z gradient magnitude = 0.005553
Atom type 1: Mean Z gradient magnitude = 0.014200
Atom type 2: Mean Z gradient magnitude = 0.002955
Atom type 3: Mean Z gradient magnitude = 0.000386
Atom type 4: Mean Z gradient magnitude = 0.000354
Atom type 0: Mean Z gradient magnitude = 0.007394
Atom type 1: Mean Z gradient magnitude = 0.014495
Atom type 2: Mean Z gradient magnitude = 0.004061
Atom type 3: Mean Z gradient magnitude = 0.000413
Atom type 4: Mean Z gradient magnitude = 0.000366
Atom type 0: Mean Z gradient magnitude = 0.006678
Atom type 1: Mean Z gradient magnitude = 0.016871
Atom type 2: Mean Z gradient magnitude = 0.002564
Atom type 3: Mean Z gradient magnitude = 0.000420
Atom type 4: Mean Z gradient magnitude = 0.000365
Atom type 0: Mean Z gradient magnitude = 0.007834
Atom type 1: Mean Z gradient magnitude = 0.016243
Atom type 2: Mean Z gradient magnitude = 0.002882
Atom type 3: Mean Z gradient magnitude = 0.000427
Atom type 4: Mean Z gradient magnitude = 0.000370


/home/huayuchen/funcmol-main-neuralfield/funcmol/gnf_visualizer.py:265: UserWarning:

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.



  GIF: gnf_reconstruction_results/recon_sample_1_predicted_field_gaussian_mag.gif
 对比图: gnf_reconstruction_results/recon_sample_1_predicted_field_gaussian_mag_final.png


In [39]:
# 可视化一维梯度场对比（不同 atom_type）
for atom_type in range(5):
    # 使用文件名包含 model_dir 信息
    model_name = os.path.basename(model_dir)
    save_path = os.path.join(output_dir, f"gradient_field_comparison_sample_0_model_{model_name}")
    gradient_results = visualize_1d_gradient_field_comparison(
        gt_coords=gt_coords,
        gt_types=gt_types,
        converter=converter,
        decoder=decoder,
        codes=codes,
        sample_idx=1, # TODO：这里可以改
        atom_type=atom_type,
        x_range=None,
        n_points=3000, # TODO：这里可以改
        y_coord=0.0,
        z_coord=0.0,
        save_path=save_path,
        gradient_field_method="gaussian_mag"  # TODO：这里可以改
    )
    if gradient_results:
        print(f"Gradient field comparison for atom_type={atom_type} (model: {model_dir}):")
        print(f"  Saved to: {gradient_results['save_path']}")

自动计算 x 轴范围: (-1.567720103263855, 1.5677199840545655)


Gradient field comparison (atom_type=C, gaussian_mag) saved to: gnf_reconstruction_results/gradient_field_comparison_sample_1_atom_C_gaussian_mag.png
Gradient field comparison for atom_type=0 (model: /home/huayuchen/funcmol-main-neuralfield/funcmol/exps/neural_field/nf_qm9_20250803_150822_593945):
  Saved to: gnf_reconstruction_results/gradient_field_comparison_sample_1_atom_C_gaussian_mag.png
自动计算 x 轴范围: (-1.567720103263855, 1.5677199840545655)
Gradient field comparison (atom_type=H, gaussian_mag) saved to: gnf_reconstruction_results/gradient_field_comparison_sample_1_atom_H_gaussian_mag.png
Gradient field comparison for atom_type=1 (model: /home/huayuchen/funcmol-main-neuralfield/funcmol/exps/neural_field/nf_qm9_20250803_150822_593945):
  Saved to: gnf_reconstruction_results/gradient_field_comparison_sample_1_atom_H_gaussian_mag.png
自动计算 x 轴范围: (-1.567720103263855, 1.5677199840545655)
Gradient field comparison (atom_type=O, gaussian_mag) saved to: gnf_reconstruction_results/gradient_